In [1]:
import requests
import json

In [2]:
%run generates_ghost_token.ipynb
token = generate_token()

In [3]:
with open('secrets.json', 'r') as file:
    json_content = json.load(file)
    
GHOST_API_URL = json_content["api_url"]

In [4]:
# Define the endpoint for uploading an image
endpoint = f'{GHOST_API_URL}images/upload/'

In [5]:
# Define the headers with the access token and content type
headers = {
    'Authorization': f'Ghost {token}',
    'Content-Type': 'multipart/form-data;',
    }

In [6]:
image_path = '/Users/42piratas/Playground/akasha/akasha-blog-migration-to-ghost/TEST1026/test.jpg'

In [7]:
# Open the image file for reading
with open(image_path, 'rb') as image_file:
    # Create a file object to send as data
    files = {'file': (image_path, image_file)}

    # Make a POST request to upload the image
    response = requests.post(endpoint, headers=headers, files=files)

    # Check if the request was successful
    if response.status_code == 201:
        # Parse the JSON response to get the image URL
        image_url = response.json()['url']
        print(image_url)
    else:
        # Handle the error
        print(f'Failed to upload image. Status code: {response.status_code}')

Failed to upload image. Status code: <Response [422]>
